In [1]:
## IMPORT NECESSARY PACKAGES
from NeuralNetConstructor import ANN

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
## LOAD AND PREPROCESS DATA
fracking = pd.read_csv(r"C:\Users\Peter\Desktop\Classes\Summer 2021\CSI Research\norm_R_table-with-taxonomy.txt", sep = "\t")
fracking_metadata = pd.read_csv(r"C:\Users\Peter\Desktop\Classes\Summer 2021\CSI Research\16S_combined_metadata.txt", sep = "\t")
combined_metadata = pd.read_csv(r"C:\Users\Peter\Desktop\Classes\Summer 2021\CSI Research\edited_mt_combined_metadata.txt", sep = "\t")
species_table = pd.read_csv(r"C:\Users\Peter\Desktop\Classes\Summer 2021\CSI Research\norm_filtered_species_table.txt", sep = "\t")

In [3]:
combined_metadata = combined_metadata.set_index("SampleID")

In [4]:
species_table = species_table.set_index("SampleID")
species_table = species_table.transpose()

In [5]:
merged_data = pd.merge(species_table, combined_metadata, left_index=True, right_index=True)

In [6]:
X = merged_data.iloc[:, :-17]
y = merged_data.loc[:, "HF_Status"]
y = np.where(y == "HF+", 1, 0)

print(X.shape, y.shape)

(99, 8218) (99,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(79, 8218) (20, 8218) (79,) (20,)


In [8]:
# initialize artificial neural network
fracking_net = ANN(input_dim = X_train.shape[1], #input dimension
                   neurons = [100, 100, 100], #number of neurons in each hidden layer
                   activation = "softplus", #activation for each layer
                   output_activation = "sigmoid", #activation for output
                   output_neurons = 1, #number of neurons in output (1 for binary classification)
                   batch_size = 32, 
                   epochs = 100,
                   use_batch_norm = True,
                   use_dropout = True,
                   dropout_rate = .4)

In [9]:
# view model architecture
fracking_net.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 8218)]            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8218)              32872     
_________________________________________________________________
layer_0 (Dense)              (None, 100)               821900    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
layer_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0     

In [13]:
#compile model
fracking_net.compile()

<bound method ANN.compile of <NeuralNetConstructor.ANN object at 0x0000013A5CFED130>>

In [11]:
# train model
fracking_net.train(X_train, y_train)

3/3 [==============================] - 0s 10ms/step - loss: 0.0895 - auc: 0.9971 - accuracy: 0.9620
Train Loss: 0.0895 
 Train AUC: 0.9971 
 Train Accuracy: 96.2025%


In [12]:
# test model
fracking_net.test(X_test, y_test)

1/1 [==============================] - 0s 27ms/step - loss: 0.0397 - auc: 1.0000 - accuracy: 1.0000
Test Loss: 0.0397 
 Test AUC: 1.0000 
 Test Accuracy: 100.0000%
